In [1]:
import os
import re
import random
import pprint
import mne
from mne.io.edf.edf import RawEDF
import numpy as np
from datetime import datetime
import pandas as pd
import pywt
from scipy.stats import skew
import math
from loading import edfloader, feature_vector

# Feature Extraction

The most discriminative features for CHB-MIT dataset (Chen et. al, 2017):

- Energy
- Max
- Min
- Mean
- Standard Deviation
- Skewness

6 features for each sub-band resulting in 6 x 6 = 36 features for channel.

## Discrete Wavelet Transformation

Discrete Wavelet Transformation (DWT) is a transform that decomposes a given signal into a number of sets, 
where each set is a time series of coefficients describing the time evolution of the signal in the corresponding frequency band




# Epochs
Since EEG-signals are continuous and non-stationary, epochs are then defined as time segments; in our case, a 2-second epoch has the dimension of 23 x 512 (sample size 256 Hz)

## Create wavelet features for 4 sec epochs of ictal-states
We try and take the middle of the signal 

In [ ]:
df = pd.read_excel('output.xlsx', index_col=0)
patient = ['chb01', 'chb24']
feature_list = []
seconds = 4
seconds = round(seconds/2)


for j in patient:
    idx_patient = df.index[(df['case'] == j)].tolist()
    
    for i in idx_patient:
        rootdir='/Users/martinnguyen/Desktop/epilepticseizures/data'
        case = '/'+df['case'][i]
        file = '/'+df['seizure_file'][i]+'.edf'
        directory = rootdir+case+file
        if directory == '/Users/martinnguyen/Desktop/epilepticseizures/data/chb24/chb24_11.edf' or directory == '/Users/martinnguyen/Desktop/epilepticseizures/data/chb24/chb24_15.edf' or directory == '/Users/martinnguyen/Desktop/epilepticseizures/data/chb24/chb24_17.edf':
            continue

        raw = edfloader(directory)

        time = round((df['seizure_start'][i]+df['seizure_end'][i])/2)
        times = [time-seconds, time + seconds]

        feature_list.append(feature_vector(raw, times))

patient = ['chb02','chb03','chb04','chb05','chb06','chb07','chb08','chb09',
         'chb10', 'chb11','chb12','chb13','chb14','chb15','chb16','chb17','chb18',
         'chb19','chb20', 'chb21', 'chb22','chb23']

for j in patient:
    idx_patient = df.index[(df['case'] == j)].tolist()
    
    for i in idx_patient:
        rootdir='/Users/martinnguyen/Desktop/epilepticseizures/data'
        case = '/'+df['case'][i]
        file = '/'+df['seizure_file'][i]+'.edf'

        directory = rootdir+case+file

        raw = edfloader(directory)

        time = round((df['seizure_start'][i]+df['seizure_end'][i])/2)
        times = [time-seconds, time + seconds]
        
        feature_list.append(feature_vector(raw, times))



In [3]:
np.save('ictal_feature.npy', feature_list)

## Create wavelet features 4 sec for inter-ictal states 
We take 60 seconds after the annotated seizure end

In [5]:
np.save('interictal_feature.npy', feature_list)

In [ ]:

df = pd.read_excel('output.xlsx', index_col=0)
patient = ['chb01', 'chb24']
feature_list = []
seconds = 4
secondsafter = 60


for j in patient:
    idx_patient = df.index[(df['case'] == j)].tolist()
    
    for i in idx_patient:
        rootdir='/Users/martinnguyen/Desktop/epilepticseizures/data'
        case = '/'+df['case'][i]
        file = '/'+df['seizure_file'][i]+'.edf'
        directory = rootdir+case+file
        seconds_afterseizure = df['seizure_end'][i]+secondsafter
        
        raw = edfloader(directory)

        times = [seconds_afterseizure, seconds_afterseizure+seconds]
        try: 
            feature_list.append(feature_vector(raw, times))
        except:
            pass


patient = ['chb02','chb03','chb04','chb05','chb06','chb07','chb08','chb09',
         'chb10', 'chb11','chb12','chb13','chb14','chb15','chb16','chb17','chb18',
         'chb19','chb20', 'chb21', 'chb22','chb23']

for j in patient:
    idx_patient = df.index[(df['case'] == j)].tolist()
    
    for i in idx_patient:
        rootdir='/Users/martinnguyen/Desktop/epilepticseizures/data'
        case = '/'+df['case'][i]
        file = '/'+df['seizure_file'][i]+'.edf'
        directory = rootdir+case+file
        seconds_afterseizure = df['seizure_end'][i]+secondsafter
        
        raw = edfloader(directory)

        times = [seconds_afterseizure, seconds_afterseizure+seconds]
        try: 
            feature_list.append(feature_vector(raw, times))
        except:
            print("LOL THIS ONE")
            pass


In [ ]:

df = pd.read_excel('output.xlsx', index_col=0)
patient = ['chb01', 'chb24']
feature_list = []
seconds = 4
secondsafter = 60


for j in patient:
    idx_patient = df.index[(df['case'] == j)].tolist()
    
    for i in idx_patient:
        rootdir='/Users/martinnguyen/Desktop/epilepticseizures/data'
        case = '/'+df['case'][i]
        file = '/'+df['seizure_file'][i]+'.edf'
        directory = rootdir+case+file
        if directory == '/Users/martinnguyen/Desktop/epilepticseizures/data/chb24/chb24_11.edf' or directory == '/Users/martinnguyen/Desktop/epilepticseizures/data/chb24/chb24_15.edf' or directory == '/Users/martinnguyen/Desktop/epilepticseizures/data/chb24/chb24_17.edf':
            continue
        seconds_afterseizure = df['seizure_end'][i]+secondsafter
        
        raw = edfloader(directory)

        times = [seconds_afterseizure, seconds_afterseizure+seconds]
        feature_list.append(feature_vector(raw, times))
       


patient = ['chb02','chb03','chb04','chb05','chb06','chb07','chb08','chb09',
         'chb10', 'chb11','chb12','chb13','chb14','chb15','chb16','chb17','chb18',
         'chb19','chb20', 'chb21', 'chb22','chb23']

for j in patient:
    idx_patient = df.index[(df['case'] == j)].tolist()
    
    for i in idx_patient:
        rootdir='/Users/martinnguyen/Desktop/epilepticseizures/data'
        case = '/'+df['case'][i]
        file = '/'+df['seizure_file'][i]+'.edf'
        directory = rootdir+case+file
        seconds_afterseizure = df['seizure_end'][i]+secondsafter
        
        raw = edfloader(directory)

        times = [seconds_afterseizure, seconds_afterseizure+seconds]
        feature_list.append(feature_vector(raw, times))

